In [ ]:
import numpy as np
import os
import tensorflow as tf

In [3]:
ACTIONS = np.array(
    [
        "hello",
        "thanks",
        "i-love-you",
        "see-you-later",
        "I",
        "Father",
    ]
)

# prepare the window of frames
window = []
for i in range(30):
    npy_path = os.path.join("../../datasets", "I", "5", f"{i}.npy")

    # load the frame data from the numpy file
    seq = np.load(npy_path)

    window.append(seq)

# convert the window to a numpy array
action = np.array(window)

In [4]:
# load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(
    model_path="../../drive/models/tflite/asl-action-cnn-lstm_1l-6a-es_p30__rlr_f05_p10_lr1e5-1.2M.tflite"
)
interpreter.allocate_tensors()

# get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# print input and output tensor details
print("Input details:", input_details)
print("Output details:", output_details)

Input details: [{'name': 'serving_default_batch_normalization_input:0', 'index': 0, 'shape': array([   1,   30, 1692]), 'shape_signature': array([  -1,   30, 1692]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall:0', 'index': 97, 'shape': array([1, 6]), 'shape_signature': array([-1,  6]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [5]:
input_details = interpreter.get_input_details()
input_shape = input_details[0]['shape']
print("Expected input shape:", input_shape)

Expected input shape: [   1   30 1692]


In [6]:
# expand dimensions to match the batch size of 1
action = action.astype(np.float32)

action = np.expand_dims(action, axis=0)

In [11]:
action

array([[[ 0.54302716,  0.47076157, -0.02970532, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.5437715 ,  0.4679922 , -0.02995613, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.546769  ,  0.46607926, -0.03016827, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.54244214,  0.4655904 , -0.03017098, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.54249424,  0.46554026, -0.03017088, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.542535  ,  0.46530834, -0.03017675, ...,  0.        ,
          0.        ,  0.        ]]], dtype=float32)

In [12]:
# Reshape the 3D array to 2D
array_2d = action.reshape(-1, action.shape[-1])

# Save the 2D array to a plain text file
np.savetxt("array_3d_as_2d.txt", array_2d)

In [14]:
# set the input tensor
interpreter.set_tensor(input_details[0]['index'], action)

# run inference
interpreter.invoke()

# get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

In [15]:
# determine the predicted action
pred = output_data[0]

print(ACTIONS[np.argmax(pred)])

I
